In [ ]:
!nvidia-smi

Sun Jan 24 16:00:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    67W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install required packages

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

In [ ]:
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [ ]:
PERCENT = 0.2 # define how much you want to use the dataset

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/NLP/')
f = open("peyma/train.txt", encoding="utf8")
lines = f.readlines()
c = 0 # Getting lines count
for line in lines:
    c += 1
c
f.close()

In [ ]:
f = open("peyma/train.txt", encoding="utf8")
training = ''
x = 0
while x < int(PERCENT * c): # using PERCENT of the dataset
    training += f.readline()
    x += 1
training[0:1000]

'مدیران|O\nچهارمین|O\nجشنواره|O\nفیلم|O\nو|O\nعکس|O\nفناوری|O\nو|O\nصنعتی|O\nمعرفی|O\nشدند|O\n\nبه|O\nگزارش|O\nخبرگزاری|B_ORG\nمهر|I_ORG\n،|O\nسعید|B_PER\nپورعلی|I_PER\nمعاون|O\nفرهنگی|O\nجهاد|B_ORG\nدانشگاهی|I_ORG\nکشور|I_ORG\nبا|O\nتاکید|O\nبر|O\nاینکه|O\nسینما|O\nیکی|O\nاز|O\nموثرترین|O\nروش های|O\nفرهنگ سازی|O\nدر|O\nحوزه|O\nصنعت|O\nو|O\nتولید|O\nاست|O\n،|O\nگفت|O\n:|O\nبا|O\nتوجه|O\nبه|O\nگستردگی|O\nروزافزون|O\nسینما|O\nو|O\nصنعت|O\nدر|O\nکشور|O\n،|O\nاین|O\nدو|O\nنمی توانند|O\nاز|O\nیکدیگر|O\nکناره گیری|O\nکنند|O\n.|O\n\nچه بسا|O\nامروز|O\nدر|O\nکشورهای|O\nپیشرفته|O\n،|O\nجایگاه|O\nفرهنگ|O\nبه|O\nعنوان|O\nیکی|O\nاز|O\nمحورهای|O\nصنعت|O\nو|O\nاقتصاد|O\nجای|O\nخود|O\nرا|O\nباز|O\nکرده|O\nاست|O\n.|O\n\nوی|O\nادامه|O\nداد|O\n:|O\nمتاسفانه|O\nبا|O\nتوجه|O\nبه|O\nساختار|O\nدولتی|O\nفرهنگ|O\nو|O\nهنر|O\nدر|O\nکشور|O\nما|O\n،|O\nاینگونه|O\nپذیرفته|O\nشده|O\nکه|O\nفرهنگ|O\nمقوله ای|O\nهزینه بر|O\nاست|O\nکه|O\nمنابع|O\nدرآمدی|O\nنامشخصی|O\nدارد|O\n.|O\n\nبه|O\nنظر|O\nمی رسد|O\nکه|O\nهمگامی

### Tokenizing

In [ ]:
data = training.split('.') # Separate by sentence
training = []
for i in range(len(data)):
    data[i] = data[i].rstrip()
    training.append(data[i].split('\n'))

In [ ]:
token = [] 
label = []
for i in range(len(training)):
    for j in range(len(training[i])):
        temp = training[i][j].split('|')
        if len(temp) == 2:
            token.append(temp[0])
            label.append(temp[1])

In [ ]:
token[13],label[13]

('خبرگزاری', 'B_ORG')

In [ ]:
len(token)

48741

In [ ]:
data = pd.DataFrame({'token':token, 'label':label})
data.head()
labels = list(sorted(data['label'].unique()))
labels

['B_DAT',
 'B_LOC',
 'B_MON',
 'B_ORG',
 'B_PCT',
 'B_PER',
 'B_TIM',
 'I_DAT',
 'I_LOC',
 'I_MON',
 'I_ORG',
 'I_PCT',
 'I_PER',
 'I_TIM',
 'O']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48741 entries, 0 to 48740
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   token   48741 non-null  object
 1   label   48741 non-null  object
dtypes: object(2)
memory usage: 761.7+ KB


In [ ]:
data.head(20)

,token,label
0,مدیران,O
1,چهارمین,O
2,جشنواره,O
3,فیلم,O
4,و,O
5,عکس,O
6,فناوری,O
7,و,O
8,صنعتی,O
9,معرفی,O


In [ ]:
fig = go.Figure()

groupby_label = data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

### Equalizing

In [ ]:
size = len(data.groupby('label')['label'].count()) # How many labels?
sep = []
temp = ''
for i in range(size):
    temp = data.groupby('label')['label'].count().index[i] # Separate based on label
    sep.append(data[data['label'] == temp])

cutting_point = min(data.groupby('label')['label'].count().tolist())
print(cutting_point)

for i in range(size):
    sep[i] = sep[i].sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat(sep)
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

41
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   token   615 non-null    object
 1   label   615 non-null    object
dtypes: object(2)
memory usage: 9.7+ KB


In [ ]:
fig = go.Figure()

groupby_label = new_data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
new_data.head()

,token,label
0,توسلی,I_PER
1,تا,I_DAT
2,اداره,B_ORG
3,میلیون,I_MON
4,جب,I_LOC


### Train Test Validation Split

In [ ]:
new_data['label_id'] = new_data['label'].apply(lambda t: labels.index(t))

train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['label'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['token'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['token'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['token'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(497, 3)
(56, 3)
(62, 3)


## TensorFlow

In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

### Configuration

In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'B_DAT': 0, 'B_LOC': 1, 'B_MON': 2, 'B_ORG': 3, 'B_PCT': 4, 'B_PER': 5, 'B_TIM': 6, 'I_DAT': 7, 'I_LOC': 8, 'I_MON': 9, 'I_ORG': 10, 'I_PCT': 11, 'I_PER': 12, 'I_TIM': 13, 'O': 14}
id2label: {0: 'B_DAT', 1: 'B_LOC', 2: 'B_MON', 3: 'B_ORG', 4: 'B_PCT', 5: 'B_PER', 6: 'B_TIM', 7: 'I_DAT', 8: 'I_LOC', 9: 'I_MON', 10: 'I_ORG', 11: 'I_PCT', 12: 'I_PER', 13: 'I_TIM', 14: 'O'}


In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B_DAT",
    "1": "B_LOC",
    "2": "B_MON",
    "3": "B_ORG",
    "4": "B_PCT",
    "5": "B_PER",
    "6": "B_TIM",
    "7": "I_DAT",
    "8": "I_LOC",
    "9": "I_MON",
    "10": "I_ORG",
    "11": "I_PCT",
    "12": "I_PER",
    "13": "I_TIM",
    "14": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B_DAT": 0,
    "B_LOC": 1,
    "B_MON": 2,
    "B_ORG": 3,
    "B_PCT": 4,
    "B_PER": 5,
    "B_TIM": 6,
    "I_DAT": 7,
    "I_LOC": 8,
    "I_MON": 9,
    "I_ORG": 10,
    "I_PCT": 11,
    "I_PER": 12,
    "I_TIM": 13,
    "O": 14
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type":

### Input Embeddings / Dataset

In [ ]:
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    
    features = glue_convert_examples_to_features(
        examples, 
        tokenizer, 
        maxlen, 
        output_mode=output_mode, 
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [ ]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=128)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_valid, y_valid, maxlen=128)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128, is_tf_dataset=False)

/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning:

This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py



In [ ]:
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'attention_mask: {value[0]["attention_mask"]}')
    print(f'token_type_ids: {value[0]["token_type_ids"]}')
    print(f'        target: {value[1]}')

     input_ids: [   2 5252    4    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
attention_mask: [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
token_type_ids: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)

    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
valid_dataset = get_training_dataset(valid_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
valid_steps = len(valid_examples) // VALID_BATCH_SIZE

train_steps, valid_steps

(31, 3)

In [ ]:
def build_model(model_name, config, learning_rate=3e-5):
    model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [ ]:
model = build_model(MODEL_NAME_OR_PATH, config, learning_rate=LEARNING_RATE)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%%time

r = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=EPOCHS,
    verbose=1)

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fe97ae00d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fe97ae00d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


31/31 [==============================] - ETA: 0s - loss: 2.6470 - accuracy: 0.1489

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


31/31 [==============================] - 65s 861ms/step - loss: 2.6418 - accuracy: 0.1527 - val_loss: 2.1224 - val_accuracy: 0.5000
Epoch 2/3
31/31 [==============================] - 25s 804ms/step - loss: 2.0507 - accuracy: 0.4902 - val_loss: 1.7088 - val_accuracy: 0.5000
Epoch 3/3
31/31 [==============================] - 25s 809ms/step - loss: 1.6152 - accuracy: 0.5213 - val_loss: 1.5626 - val_accuracy: 0.4167
FINAL ACCURACY MEAN:  0.4722222189108531
CPU times: user 39.1 s, sys: 7.78 s, total: 46.9 s
Wall time: 1min 54s


In [ ]:
# save the model

model.save_pretrained(os.path.dirname(OUTPUT_PATH))

In [ ]:
ev = model.evaluate(test_dataset_base.batch(TEST_BATCH_SIZE))
print()
print(f'Evaluation: {ev}')
print()

predictions = model.predict(xtest)
ypred = predictions[0].argmax(axis=-1).tolist()

print()
print(classification_report(ytest, ypred, target_names=labels))
print()

print(f'F1: {f1_score(ytest, ypred, average="weighted")}')

4/4 [==============================] - 1s 244ms/step - loss: 1.4182 - accuracy: 0.6129


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.



Evaluation: [1.4181872606277466, 0.6129032373428345]


              precision    recall  f1-score   support

       B_DAT       0.50      0.25      0.33         4
       B_LOC       0.67      0.50      0.57         4
       B_MON       1.00      0.25      0.40         4
       B_ORG       0.80      1.00      0.89         4
       B_PCT       0.36      1.00      0.53         4
       B_PER       0.42      1.00      0.59         5
       B_TIM       0.80      1.00      0.89         4
       I_DAT       0.50      0.25      0.33         4
       I_LOC       0.50      0.25      0.33         4
       I_MON       1.00      0.25      0.40         4
       I_ORG       1.00      0.25      0.40         4
       I_PCT       1.00      1.00      1.00         5
       I_PER       1.00      0.25      0.40         4
       I_TIM       0.60      0.75      0.67         4
           O       0.67      1.00      0.80         4

    accuracy                           0.61        62
   macro avg       0.72 